# Clustering Homework Assignment
Matthew Paras

1/14/2019

Data found at: https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Physician-and-Other-Supplier2016.html

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
# Read in the data
c_size = 10000000
data = pd.read_csv('Medicare_Provider_Util_Payment_PUF_CY2016.txt', sep = "\t", chunksize=c_size)

# for now, load in just the first 10000 lines of the data frame
# data is an iterator so I would need to concatenate varying sizes if I dont want all of the data at one time
output = pd.DataFrame(data.get_chunk(c_size))
output = output.iloc[1:]
output = output.reset_index(drop=True)

In [ ]:
# Peer into the data
output.head()

### Feature Demographics

In [ ]:
# HCPCS_CODE Mapping to Description
hcpcs_codes = output[['HCPCS_CODE', 'HCPCS_DESCRIPTION']].drop_duplicates()

# Generate Demographics

# Number of different providers
print("Number of unique providers: ", output['NPI'].nunique())

# Number of different cities
print("Number of different cities represented: ", output['NPPES_PROVIDER_CITY'].nunique())
      
# Number of different HCPCS_codes:
print("Number of different HCPCS_codes: ", output['HCPCS_CODE'].nunique())

# Gender Demographics
print("Gender Demographics: ")
print(output['NPPES_PROVIDER_GENDER'].value_counts())

# Credentials Demographics
print("Creditionals Demographics: ")
print(output['NPPES_CREDENTIALS'].value_counts())

# Entity Code Demographics
print("Entity Code Demographics: ")
print(output['NPPES_ENTITY_CODE'].value_counts())

## Feature Histograms

In [ ]:
numeric_data = output._get_numeric_data()
column_names = list(numeric_data)[2:]

for column_name in column_names:
    # plt.figure(figsize=(10,5))
    # find percentile
    upper_bound = np.percentile(numeric_data[column_name], 95)
    plt.hist(numeric_data[column_name], range = (0, upper_bound))
    plt.xlabel(column_name)
    plt.ylabel("Amount $")
    plt.show()

In [ ]:
# scale the data (currently only scale the numeric data)
mms = MinMaxScaler()
numeric_data = output._get_numeric_data()
df_scaled = pd.DataFrame(mms.fit_transform(numeric_data), columns = numeric_data.columns)
df_scaled = df_scaled[df_scaled.columns[2:]]

## Elbow method for determining the optimal number of clusters

In [ ]:
# Elbow method for determining the optimal number of clusters
Sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(df_scaled)
    Sum_of_squared_distances.append(km.inertia_)
    
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

## Silhouette Analysis

Adapted from: https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html

In [ ]:
# Silhouette Analysis here
from __future__ import print_function

# from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np


X = df_scaled.values
# y = model.labels_


range_n_clusters = range(2, 10)

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()

In [ ]:
# K Means Cluster
model = KMeans(n_clusters=8)
model.fit(df_scaled)

In [ ]:
# Dimensionality reduction to visualize the clustering
 
# Plot the Models Classifications
pca = PCA(2)  # project from x to 2 dimensions
projected = pca.fit_transform(df_scaled)
plt.scatter(projected[:, 0], projected[:, 1], c=model.labels_, edgecolor='none', alpha=0.5)
plt.show()

## Centers of Labeled Data

In [ ]:
# look at demographics of now labeled data
# Don't do this copy theres not enough memory
labeled_data = output.copy()
numeric_data = output._get_numeric_data()
column_names = list(numeric_data)[2:]
labeled_data = labeled_data[column_names]
labeled_data['Label'] = model.labels_

labeled_data.groupby(['Label']).mean()

In [ ]:
# Generate Demographics
# Don't do this copy here there isn't enough memory
labeled_data = output.copy()
labeled_data['Label'] = model.labels_

for index, row in labeled_data.iterrows():
    print(row['Label'])

# for i in range(8):
#     value_counts = labeled_data[labeled_data['Label'] == 0]['NPPES_PROVIDER_GENDER'].value_counts()
#     print("Cluster", i, ": ", value_counts)